# Tutorial 5: Neural Architecture Search (NAS) with Mase and Optuna

In this tutorial, we'll see how Mase can be integrated with Optuna, the popular hyperparameter optimization framework, to search for a Bert model optimized for sequence classification on the IMDb dataset. We'll take the Optuna-generated model and import it into Mase, then run the CompressionPipeline to prepare the model for edge deployment by quantizing and pruning its weights.

As we'll see, running Architecture Search with Mase/Optuna involves the following steps.

1. **Define the search space**: this is a dictionary containing the range of values for each parameter at each layer in the model.

2. **Write the model constructor**: this is a function which uses Optuna utilities to sample a model from the search space, and constructs the model using transformers from_config class method.

3. **Write the objective function**: this function calls on the model constructor defined in Step 2 and defines the training/evaluation setup for each search iteration.

4. **Go!** Choose an Optuna sampler, create a study and launch the search.

In [1]:
checkpoint = "prajjwal1/bert-tiny"
tokenizer_checkpoint = "bert-base-uncased"
dataset_name = "imdb"

First, fetch the dataset using the `get_tokenized_dataset` utility.

In [2]:
from chop.tools import get_tokenized_dataset

dataset, tokenizer = get_tokenized_dataset(
    dataset=dataset_name,
    checkpoint=tokenizer_checkpoint,
    return_tokenizer=True,
)

/vol/bitbucket/nr722/adls_group_14/.venv/lib/python3.11/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/vol/bitbucket/nr722/adls_group_14/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO     Tokenizing dataset imdb with AutoTokenizer for bert-base-uncased.


## 1. Defining the Search Space

We'll start by defining a search space, i.e. enumerating the possible combinations of hyperparameters that Optuna can choose during search. We'll explore the following range of values for the model's hidden size, intermediate size, number of layers and number of heads, inspired by the [NAS-BERT paper](https://arxiv.org/abs/2105.14444).

In [3]:
import torch.nn as nn
from chop.nn.modules import Identity

search_space = {
    "num_layers": [2, 4, 8],
    "num_heads": [2, 4, 8, 16],
    # hidden size is the embedding dimension in transformers
    "hidden_size": [128, 192, 256, 384, 512],
    # intermediate size is the dimension of the feedforward layer in transformers
    "intermediate_size": [512, 768, 1024, 1536, 2048],
    "linear_layer_choices": [
        "Linear",
        "Identity",
    ],
}

## 2. Writing a Model Constructor

We define the following function, which will get called in each iteration of the search process. The function is passed the `trial` argument, which is an Optuna object that comes with many functionalities - see the [Trial documentation](https://optuna.readthedocs.io/en/stable/reference/trial.html) for more details. Here, we use the `trial.suggest_int` and `trial.suggest_categorical` functions to trigger the chosen sampler to choose parameter choices and layer types. The suggested integer is the index into the search space for each parameter, which we defined in the previous cell.

In [4]:
from transformers import AutoConfig, AutoModelForSequenceClassification
from chop.tools.utils import deepsetattr

def construct_model(trial):
    config = AutoConfig.from_pretrained(checkpoint)

    # Update the paramaters in the config
    for param in [
        "num_layers",
        "num_heads",
        "hidden_size",
        "intermediate_size",
    ]:
        chosen_idx = trial.suggest_int(param, 0, len(search_space[param]) - 1)
        setattr(config, param, search_space[param][chosen_idx])

    trial_model = AutoModelForSequenceClassification.from_config(config)

    # Map string choices to actual classes
    layer_map = {
        "Linear": nn.Linear,
        "Identity": Identity
    }

    for name, layer in trial_model.named_modules():
        if isinstance(layer, nn.Linear) and layer.in_features == layer.out_features:
            # trial.suggest_categorical returns a string now (e.g. "Linear" or "Identity")
            new_layer_name = trial.suggest_categorical(
                f"{name}_type",
                search_space["linear_layer_choices"],
            )

            new_layer_cls = layer_map[new_layer_name]

            if new_layer_cls == nn.Linear:
                continue
            elif new_layer_cls == Identity:
                new_layer = Identity()
                deepsetattr(trial_model, name, new_layer)
            else:
                raise ValueError(f"Unknown layer type: {new_layer_cls}")

    return trial_model

## 3. Defining the Objective Function

Next, we define the objective function for the search, which gets called on each trial. In each trial, we create a new model instace with chosen hyperparameters according to the defined sampler. We then use the `get_trainer` utility in Mase to run a training loop on the IMDb dataset for a number of epochs. Finally, we use `evaluate` to report back the classification accuracy on the test split.

In [5]:
from chop.tools import get_trainer
import torch

def objective(trial):
    
    # Define the model and move to GPU
    model = construct_model(trial)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    trainer = get_trainer(
        model=model,
        tokenized_dataset=dataset,
        tokenizer=tokenizer,
        evaluate_metric="accuracy",
        num_train_epochs=1,
    )

    trainer.train()
    eval_results = trainer.evaluate()

    # Move back to CPU for storage
    trial.set_user_attr("model", model.cpu())

    return eval_results["eval_accuracy"]

## 4. Launching the Search

Optuna provides a number of samplers, for example:

* **GridSampler**: iterates through every possible combination of hyperparameters in the search space
* **RandomSampler**: chooses a random combination of hyperparameters in each iteration
* **TPESampler**: uses Tree-structured Parzen Estimator algorithm to choose hyperparameter values.

You can define the chosen sampler by simply importing from `optuna.samplers` as below.

In [6]:
from optuna.samplers import GridSampler, RandomSampler, TPESampler

sampler = RandomSampler()

With all the pieces in place, we can launch the search as follows. The number of trials is set to 1 so you can go get a coffee for 10 minutes, then proceed with the tutorial. However, this will essentially be a random model - for better results, set this to 100 and leave it running overnight!

In [ ]:
import optuna

study = optuna.create_study(
    direction="maximize",
    study_name="bert-tiny-nas-study",
    sampler=sampler,
)

study.optimize(
    objective,
    n_trials=1,
    timeout=60 * 60 * 24,
)

Fetch the model associated with the best trial as follows, and export to be used in future tutorials. In Tutorial 6, we'll see how to run mixed-precision quantization search on top of the model we've just found through NAS to further find the optimal quantization mapping.

In [ ]:
from pathlib import Path
import dill

model = study.best_trial.user_attrs["model"].cpu()

with open(f"{Path.home()}/tutorial_5_best_model.pkl", "wb") as f:
    dill.dump(model, f)

## Deploying the Optimized Model with CompressionPipeline

Now, we can run the CompressionPipeline in Mase to run uniform quantization and pruning over the searched model.

In [ ]:
from chop.pipelines import CompressionPipeline
from chop import MaseGraph

mg = MaseGraph(model)
pipe = CompressionPipeline()

quantization_config = {
    "by": "type",
    "default": {
        "config": {
            "name": None,
        }
    },
    "linear": {
        "config": {
            "name": "integer",
            # data
            "data_in_width": 8,
            "data_in_frac_width": 4,
            # weight
            "weight_width": 8,
            "weight_frac_width": 4,
            # bias
            "bias_width": 8,
            "bias_frac_width": 4,
        }
    },
}

pruning_config = {
    "weight": {
        "sparsity": 0.5,
        "method": "l1-norm",
        "scope": "local",
    },
    "activation": {
        "sparsity": 0.5,
        "method": "l1-norm",
        "scope": "local",
    },
}

mg, _ = pipe(
    mg,
    pass_args={
        "quantize_transform_pass": quantization_config,
        "prune_transform_pass": pruning_config,
    },
)

Finally, export the MaseGraph for the compressed checkpoint to be used in future tutorials for hardware generation and distributed deployment.

In [ ]:
mg.export(f"{Path.home()}/tutorial_5_nas_compressed", save_format="state_dict")

In [ ]:
search_space = {
    "num_layers": [2, 4, 8],
    "num_heads": [2, 4, 8, 16],
    # hidden size is the embedding dimension in transformers
    "hidden_size": [128, 192, 256, 384, 512],
    # intermediate size is the dimension of the feedforward layer in transformers
    "intermediate_size": [512, 768, 1024, 1536, 2048],
    "linear_layer_choices": [
        nn.Linear,
        # Identity,
    ],
}
# search_space = {
#     "num_layers": [2, ],
#     "num_heads": [2, 4],
#     # hidden size is the embedding dimension in transformers
#     "hidden_size": [128, 512],
#     # intermediate size is the dimension of the feedforward layer in transformers
#     "intermediate_size": [512],
#     "linear_layer_choices": [
#         nn.Linear,
#         Identity,
#     ],
# }

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import optuna
from optuna.samplers import RandomSampler, TPESampler, GridSampler
import torch.nn as nn
from chop.nn.modules import Identity
import traceback
import warnings

# Suppress warnings
warnings.filterwarnings("ignore", category=UserWarning, module="optuna.distributions")

# --- 1. FULL Search Space (for Random/TPE) ---
search_space = {
    "num_layers": [2, 4, 8],
    "num_heads": [2, 4, 8, 16],
    "hidden_size": [128, 192, 256, 384, 512],
    "intermediate_size": [512, 768, 1024, 1536, 2048],
    "linear_layer_choices": ["Linear", "Identity"],
}

# --- 2. FIXED Search Space (for GridSampler) ---
# CRITICAL: construct_model uses `trial.suggest_int` which returns an index. 
# Therefore, GridSampler must search over INDICES (0, 1, 2...), not raw values (2, 4, 8...).
grid_search_space = {
    "num_layers": list(range(len(search_space["num_layers"]))),
    "num_heads": list(range(len(search_space["num_heads"]))),
    "hidden_size": list(range(len(search_space["hidden_size"]))),
    "intermediate_size": list(range(len(search_space["intermediate_size"]))),
}

print("DEBUG: grid_search_space['num_layers'] is:", grid_search_space['num_layers'])

# We must pre-populate the grid with ALL possible layer types for ALL possible layers (up to max layers=8).
max_layers = 8
for i in range(max_layers):
    for sublayer in ["attention.self.query", "attention.self.key", "attention.self.value", 
                     "attention.output.dense", "intermediate.dense", "output.dense"]:
        # The key name must match exactly what `construct_model` asks for: e.g. "bert.encoder.layer.0.output.dense_type"
        param_name = f"bert.encoder.layer.{i}.{sublayer}_type"
        
        # For GridSampler we fix this to just "Linear" to avoid combinatorial explosion
        # grid_search_space[param_name] = ["Linear", "Identity"]
        grid_search_space[param_name] = ["Linear"]

# FIX: The BERT Pooler layer (bert.pooler.dense) is also a Linear layer with in_features == out_features.
# It gets picked up by construct_model, so it MUST be in the grid.
grid_search_space["bert.pooler.dense_type"] = ["Linear"]


# --- 3. Sampler Definition ---
samplers = {
    # We pass the fully-populated static grid to GridSampler
    "Grid Search": GridSampler(grid_search_space),
    "Random Search": RandomSampler(),
    "TPE (Bayesian)": TPESampler(),
}

n_trials = 30
sampler_history = {}
best_overall_value = -float("inf")
best_overall_model = None

# --- 4. Optimization Loop ---
for name, sampler in samplers.items():
    print(f"--- Starting {name} ---")
    try:
        # Standard Optuna setup
        study = optuna.create_study(direction="maximize", sampler=sampler)
        study.optimize(objective, n_trials=n_trials)
        
        values = [t.value for t in study.trials if t.value is not None]
        if values:
            sampler_history[name] = np.maximum.accumulate(values)
            
            if study.best_value > best_overall_value:
                best_overall_value = study.best_value
                best_overall_model = study.best_trial.user_attrs["model"]
                
    except Exception as e:
        # traceback.print_exc()
        print(f"Sampler {name} encountered an error: {e}")

# Plotting
plt.figure(figsize=(10, 6))
for name, best_values in sampler_history.items():
    if name in sampler_history:
        plt.plot(range(1, len(sampler_history[name]) + 1), sampler_history[name], marker="o", label=name)

plt.xlabel("Number of Trials")
plt.ylabel("Maximum Accuracy Achieved")
plt.title("NAS Sampler Comparison (GPU-Accelerated)")
plt.legend()
plt.grid(True)
plt.show()

: 

In [7]:
from transformers import AutoModel
import torch.nn as nn

# Load the base model to inspect its architecture
checkpoint = "prajjwal1/bert-tiny"
model = AutoModel.from_pretrained(checkpoint)

# Get the first encoder layer (BertLayer)
encoder_block = model.encoder.layer[0]

print("--- Linear Layers in one BERT Encoder Block ---")
count = 0
for name, module in encoder_block.named_modules():
    # Filter for Linear layers
    if isinstance(module, nn.Linear):
        # We print the name relative to the block
        print(f"{count+1}. {name}")
        count += 1

print(f"\nVerified Total: {count} Linear Layers per Block")

--- Linear Layers in one BERT Encoder Block ---
1. attention.self.query
2. attention.self.key
3. attention.self.value
4. attention.output.dense
5. intermediate.dense
6. output.dense

Verified Total: 6 Linear Layers per Block


In [8]:
import optuna
from optuna.samplers import RandomSampler
import torch.nn as nn
from chop.nn.modules import Identity

print("--- Verifying Sub-Layer Types in Randomly Sampled Models ---")
print("(Checking Layer 0 to confirm Identity/Linear mixing)")

def verify_sublayers(trial):
    # Construct a model from the search space
    model = construct_model(trial)
    
    # Inspect Layer 0 of the encoder to see what types were chosen
    if hasattr(model, 'bert'):
        layer0 = model.bert.encoder.layer[0]
    else:
        # Fallback for models that might be wrapped differently or different architectures
        layer0 = model.encoder.layer[0] 
    
    # Dictionary of the 6 sub-layers we expect to be swappable
    sublayers = {
        "Query": layer0.attention.self.query,
        "Key": layer0.attention.self.key,
        "Value": layer0.attention.self.value,
        "Attn Output": layer0.attention.output.dense,
        "Intermediate": layer0.intermediate.dense,
        "Output": layer0.output.dense
    }
    
    print(f"\nTrial {trial.number} Choices (Layer 0):")
    for name, module in sublayers.items():
        # Identify the type
        type_name = "Identity" if isinstance(module, Identity) else "Linear"
        print(f"  {name:12s}: {type_name}")

    return 0

# We use RandomSampler because GridSampler is restricted to Linear-only in our setup
study_check = optuna.create_study(sampler=RandomSampler())
study_check.optimize(verify_sublayers, n_trials=3)

[I 2026-02-02 15:27:43,738] A new study created in memory with name: no-name-0b067e3a-9399-4b0e-a2e7-0ed4b7560354
[I 2026-02-02 15:27:43,903] Trial 0 finished with value: 0.0 and parameters: {'num_layers': 0, 'num_heads': 1, 'hidden_size': 1, 'intermediate_size': 4, 'bert.encoder.layer.0.attention.self.query_type': 'Identity', 'bert.encoder.layer.0.attention.self.key_type': 'Identity', 'bert.encoder.layer.0.attention.self.value_type': 'Linear', 'bert.encoder.layer.0.attention.output.dense_type': 'Linear', 'bert.encoder.layer.1.attention.self.query_type': 'Identity', 'bert.encoder.layer.1.attention.self.key_type': 'Linear', 'bert.encoder.layer.1.attention.self.value_type': 'Linear', 'bert.encoder.layer.1.attention.output.dense_type': 'Identity', 'bert.pooler.dense_type': 'Linear'}. Best is trial 0 with value: 0.0.


--- Verifying Sub-Layer Types in Randomly Sampled Models ---
(Checking Layer 0 to confirm Identity/Linear mixing)

Trial 0 Choices (Layer 0):
  Query       : Identity
  Key         : Identity
  Value       : Linear
  Attn Output : Linear
  Intermediate: Linear
  Output      : Linear


[I 2026-02-02 15:27:44,160] Trial 1 finished with value: 0.0 and parameters: {'num_layers': 2, 'num_heads': 2, 'hidden_size': 4, 'intermediate_size': 0, 'bert.encoder.layer.0.attention.self.query_type': 'Identity', 'bert.encoder.layer.0.attention.self.key_type': 'Identity', 'bert.encoder.layer.0.attention.self.value_type': 'Linear', 'bert.encoder.layer.0.attention.output.dense_type': 'Linear', 'bert.encoder.layer.0.intermediate.dense_type': 'Identity', 'bert.encoder.layer.0.output.dense_type': 'Identity', 'bert.encoder.layer.1.attention.self.query_type': 'Identity', 'bert.encoder.layer.1.attention.self.key_type': 'Identity', 'bert.encoder.layer.1.attention.self.value_type': 'Linear', 'bert.encoder.layer.1.attention.output.dense_type': 'Linear', 'bert.encoder.layer.1.intermediate.dense_type': 'Linear', 'bert.encoder.layer.1.output.dense_type': 'Linear', 'bert.pooler.dense_type': 'Identity'}. Best is trial 0 with value: 0.0.
[I 2026-02-02 15:27:44,298] Trial 2 finished with value: 0.0 an


Trial 1 Choices (Layer 0):
  Query       : Identity
  Key         : Identity
  Value       : Linear
  Attn Output : Linear
  Intermediate: Identity
  Output      : Identity

Trial 2 Choices (Layer 0):
  Query       : Linear
  Key         : Identity
  Value       : Identity
  Attn Output : Identity
  Intermediate: Linear
  Output      : Linear
